# Building an adjacency matrix from a 2-D grid

Suppose we have the following 2x2 grid (map) represented by a two-dimensional array:

In [15]:
import numpy as np

just_a_grid = np.array([["a", "b"],
                        ["c", "d"]])

At the moment, this is not a graph as there are no rules to tell us which other nodes each node can reach.

We are going to apply a rule to transform our array into a **unweighted**, **directional** graph.

Our rule will be:

*Nodes can reach directly neighbouring nodes (immediately left, right, above or below) if the neighbouring node is lower, equal to or one place higher in the alphabet than the current node. The current node can reach itself.*

### Examples

- a can reach a, b
- b can reach a, b, c
- c can reach a, b, c, d

Suppose we have the following arrangement of nodes:

```
a b
c d
```

The goal is to construct an [adjacency matrix](https://en.wikipedia.org/wiki/Adjacency_matrix) that tells us whether each node on a directed graph is reachable from the current node.

Our adjacency matrix will be the square of the dimensions of our original grid—each node has a relationship with each other node in the grid.

The first column in the adjacency matrix based on the 2x2 grid above (and our rule) we would expect to see is:

```
1
1
0
0
```

Which says: `a` can reach itself and `b`, but cannot reach `c` or `d`.

Completing the matrix we get:

```
1 1 1 0
1 1 0 1
0 0 1 1
0 0 1 1
```

For larger grids it should be obvious that the adjacency matrix will end up being mostly zeroes as the most other nodes a node can reach is four.

We can create a function to take a grid and rule and return the resultant adjacency matrix.

Rough outline of the function:
- Take a NumPy array (the grid) and a comparator function as arguments
- Create a zeroed Pandas DataFrame (the adjacency matrix)
- Loop through each row in the grid
- Loop through each node in each row
- For each node, determine if itself, the nodes above, below, to the left and to the right can be reached based on the comparator function that was passed in and, if so, update the appropriate position in the adjacency matrix with a `1`

The function might look something like:

In [16]:
from typing import Callable
import pandas as pd


def build_adjacency_matrix(grid: np.array, reachability_calculator: Callable) -> pd.DataFrame:
    height = len(grid)
    width = grid.size / height
    adjacency_matrix = pd.DataFrame(0, columns=grid.flatten(), index=grid.flatten())
    np.fill_diagonal(adjacency_matrix.values, 1)
    for row_idx, row in enumerate(grid):
        for col_idx, node in enumerate(row):
            if reachability_calculator(node, node):
                adjacency_matrix[node][node] = 1
            if col_idx != 0:
                node_to_left = grid[row_idx, col_idx - 1]
                if reachability_calculator(node, node_to_left):
                    adjacency_matrix[node][node_to_left] = 1
            if col_idx < width - 1:
                node_to_right = grid[row_idx, col_idx + 1]
                if reachability_calculator(node, node_to_right):
                    adjacency_matrix[node][node_to_right] = 1
            if row_idx != 0:
                node_above = grid[row_idx - 1, col_idx]
                if reachability_calculator(node, node_above):
                    adjacency_matrix[node][node_above] = 1
            if row_idx < height - 1:
                node_below = grid[row_idx + 1, col_idx]
                if reachability_calculator(node, node_below):
                    adjacency_matrix[node][node_below] = 1

    return adjacency_matrix

And our `reachability_calculator` function, based on our rule above might be:

In [17]:
def can_reach(current_val, neighbour_val):
    return ord(current_val) + 1 >= ord(neighbour_val)

Testing our function:

In [18]:
import unittest


class TestNotebook(unittest.TestCase):

    def test_constructing_adjacency_matrix(self):
        grid = np.array([["a", "b"],
                         ["c", "d"]])

        assert build_adjacency_matrix(grid=grid, reachability_calculator=can_reach).equals(pd.DataFrame([
            [1, 1, 1, 0],
            [1, 1, 0, 1],
            [0, 0, 1, 1],
            [0, 0, 1, 1],
        ], columns=grid.flatten(), index=grid.flatten()))


unittest.main(argv=[''], verbosity=2, exit=False)

test_constructing_adjacency_matrix (__main__.TestNotebook.test_constructing_adjacency_matrix) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.006s

OK


Applying our function to a larger grid, we see that the adjacency matrix quickly becomes very large and, as mentioned above, is pretty sparse:


In [19]:
slightly_larger_grid = np.array([["a", "b", "c"],
                                 ["d", "e", "f"],
                                 ["g", "h", "i"]])

adj_matrix = build_adjacency_matrix(grid=slightly_larger_grid, reachability_calculator=can_reach)

print(adj_matrix)

   a  b  c  d  e  f  g  h  i
a  1  1  0  1  0  0  0  0  0
b  1  1  1  0  1  0  0  0  0
c  0  1  1  0  0  1  0  0  0
d  0  0  0  1  1  0  1  0  0
e  0  0  0  1  1  1  0  1  0
f  0  0  0  0  1  1  0  0  1
g  0  0  0  0  0  0  1  1  0
h  0  0  0  0  0  0  1  1  1
i  0  0  0  0  0  0  0  1  1


Also note that for simplicity we have just named the columns and row indices according to the letter of the node. A more sophisticated naming strategy would be required for anything more complex.